In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

Finding percentage of the documents in `spam_data` that are spam.

In [3]:
def answer_one():
    
    
    return np.mean(spam_data['target'])*100

In [4]:
answer_one()

13.406317300789663


Fitting the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    
    return max(vect.get_feature_names(),key=len) 

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

Fitting and transforming the training data `X_train` using a Count Vectorizer with default parameters.
Fitting a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`and finding the area under the curve (AUC) score using the transformed test data.


In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    clf = MultinomialNB(alpha=0.1).fit(X_train_vect,y_train)
    pred = clf.predict(vect.transform(X_test))
    
    
    return roc_auc_score(y_test,pred)

In [8]:
answer_three()

0.97208121827411165


Fitting and transforming the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Putting these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series is feature name, and the data is the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs are sorted largest first. 


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vect = TfidfVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    features = np.array(vect.get_feature_names())
    feature_tfidf = X_train_vect.max(0).toarray()[0]
    df = pd.DataFrame({'feature_name':features,'tfidf':feature_tfidf})
    
    asc = df.sort_values(by=['tfidf','feature_name'],ascending=(True,True))
    smallest = pd.Series(asc['tfidf'][0:20])
    smallest.index = asc['feature_name'][0:20]
    #print(smallest)
    dsc = df.sort_values(by=['tfidf','feature_name'],ascending=(False,True))
    #print(dsc)
    biggest = pd.Series(dsc['tfidf'][0:20])
    biggest.index=dsc['feature_name'][0:20]
    
    return (smallest,biggest)

In [10]:
answer_four()


(feature_name
 aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 Name: tfidf, dtype: float64, feature_name
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick    


Fitting and transforming the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fitting a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and computing the area under the curve (AUC) score using the transformed test data.


In [11]:
def answer_five():
    
    vect = TfidfVectorizer(min_df = 3).fit(X_train)
    X_train_vect = vect.transform(X_train)
    clf = MultinomialNB(alpha=0.1).fit(X_train_vect,y_train)
    pred_y = clf.predict(vect.transform(X_test))
    return roc_auc_score(y_test,pred_y)
    

In [12]:
answer_five()

0.94162436548223349

Finding average length of documents (number of characters) for not spam and spam documents?


In [13]:
def answer_six():
    
    spam = spam_data[spam_data['target'] == 1]
    not_spam = spam_data[spam_data['target'] == 0]
    
    return (np.mean(not_spam['text'].str.len()),np.mean(spam['text'].str.len()))

In [14]:
answer_six()

(71.023626943005183, 138.8661311914324)

In [121]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

Fitting and transforming the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fitting a Support Vector Classification model with regularization `C=10000`. Then computing the area under the curve (AUC) score using the transformed test data.


In [16]:
from sklearn.svm import SVC

def answer_seven():
    
    vect=TfidfVectorizer(min_df = 5).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_train_new = add_feature(X_train_vect,(X_train.str.len()))
    svm = SVC(C=10000).fit(X_train_new,y_train)
    preds = svm.predict(add_feature(vect.transform(X_test),X_test.str.len()))
    
    
    return roc_auc_score(y_test,preds)

In [17]:
answer_seven()

0.95813668234215565

Finding the average number of digits per document for not spam and spam documents.


In [39]:
def answer_eight():
    
    spam = spam_data[spam_data['target']==1]
    not_spam = spam_data[spam_data['target']==0]
    
    spam_mean = np.mean(spam.apply(lambda x: sum([c.isdigit() for c in (x['text'])]) , axis=1))
    non_spam_mean = np.mean(not_spam.apply(lambda x: sum([c.isdigit() for c in (x['text'])]) , axis=1))
    
    return (non_spam_mean,spam_mean)

In [40]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

Fitting and transforming the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

Fitting a Logistic Regression model with regularization `C=100` and computing the area under the curve (AUC) score using the transformed test data.


In [118]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5,ngram_range=(1,3)).fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_train_new = add_feature(X_train_vect,X_train.str.len())
    X_train_new = add_feature(X_train_new,X_train.apply(lambda x: sum([c.isdigit() for c in x])))
    clf = LogisticRegression(C=100).fit(X_train_new,y_train)
    X_test_vect = vect.transform(X_test)
    X_test_new = add_feature(X_test_vect,X_test.str.len())
    X_test_new = add_feature(X_test_new,X_test.apply(lambda x: sum([c.isdigit() for c in x])))
    
    pred = clf.predict(X_test_new)
    return roc_auc_score(y_test,pred)
    

In [48]:
answer_nine()

0.96533283533945646

Finding the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents.


In [88]:
def answer_ten():
    import re
    spam = spam_data[spam_data['target']==1]
    not_spam = spam_data[spam_data['target']==0]
    
    spm = ((len(spam['text'].str.extractall('([^\d^\w])')))/len(spam))
    nspm = ((len(not_spam['text'].str.extractall('([^\d^\w])')))/len(not_spam))
    
    return (nspm,spm)

In [90]:
answer_ten()


[' ', ' ', ' ', '@', '!', '&', '*']


Fitting and transforming the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

Fitting a Logistic Regression model with regularization C=100. Then computing the area under the curve (AUC) score using the transformed test data.


In [146]:
def answer_eleven():
    import re
    
    vect = CountVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train)
    X_train_vect = vect.transform(X_train)
    X_train_new = add_feature(X_train_vect,pd.Series(X_train.str.len(),name='length_of_doc'))
    X_train_new = add_feature(X_train_new,pd.Series(X_train.apply(lambda x: sum([c.isdigit() for c in x])),name='digit_count'))
    X_train_new = add_feature(X_train_new,pd.Series(X_train.apply(lambda x: len(re.findall('([^\d^\w])',x))),name='non_word_char_count'))
    
    clf = LogisticRegression(C=100).fit(X_train_new,y_train)
    
    X_test_vect = vect.transform(X_test)
    X_test_new = add_feature(X_test_vect,pd.Series(X_test.str.len(),name='length_of_doc'))
    X_test_new = add_feature(X_test_new,pd.Series(X_test.apply(lambda x: sum([c.isdigit() for c in x])),name='digit_count'))
    X_test_new = add_feature(X_test_new,pd.Series(X_test.apply(lambda x: len(re.findall('([^\d^\w])',x))),name='non_word_char_count'))
    
    pred = clf.predict(X_test_new)
    score = roc_auc_score(y_test,pred)
    
    feature_names = np.array(vect.get_feature_names())
    sorted_coef_index = clf.coef_[0].argsort()

    smallest = list(feature_names[sorted_coef_index[:10]])
    largest = list(feature_names[sorted_coef_index[-11:-1]])
    return (score,smallest,largest)

In [147]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['eply ', 'ar', 'ww', ' x', 'mob', ' ch', 'xt', 'co', 'ia', 'ne'])